In [38]:
import dedupe
import csv

In [39]:
f=open('sample_input','r')  #training dataset
c=csv.DictReader(f)

In [40]:
row=1
data={}
for line in c:
    data[row]=line
    row+=1
    
f.close()

### First ten items:

In [87]:
data.items()[:10]

[(1, {'dob': '01-03-1968', 'fn': 'WILLIAM', 'gn': 'F', 'ln': 'SMITH JR'}),
 (2, {'dob': '01-03-1968', 'fn': 'WILLIAM', 'gn': 'F', 'ln': 'ROTHMEYER JR'}),
 (3, {'dob': '01-03-1968', 'fn': 'WILLIAM', 'gn': 'F', 'ln': 'ASBY JR'}),
 (4, {'dob': '01-03-1968', 'fn': 'WILLIAM', 'gn': 'F', 'ln': 'SALTER JR'}),
 (5, {'dob': '01-03-1968', 'fn': 'WILLIAM', 'gn': 'F', 'ln': 'SALTER JR'}),
 (6, {'dob': '21-02-1962', 'fn': 'WILLIAM', 'gn': 'F', 'ln': 'BLAND JR'}),
 (7, {'dob': '21-02-1962', 'fn': 'WILLIAM', 'gn': 'F', 'ln': 'BLAND JR'}),
 (8, {'dob': '21-02-1962', 'fn': 'WILLIAM', 'gn': 'F', 'ln': 'BLAND JR'}),
 (9, {'dob': '21-02-1962', 'fn': 'WILLIAM', 'gn': 'F', 'ln': 'SHAFFER JR'}),
 (10, {'dob': '21-02-1962', 'fn': 'WILLIAM', 'gn': 'F', 'ln': 'SHAFFER JR'})]

In [42]:
variables=[
             {'field' : 'ln', 'type': 'String'},
             {'field' : 'dob', 'type': 'String'},
             {'field' : 'fn', 'type': 'String'},
             {'field' : 'gn', 'type': 'String'}
             ]

In [43]:
deduper = dedupe.Dedupe(variables)

In [44]:
deduper.sample(data,sample_size=70)

c:\users\user\miniconda2\lib\site-packages\dedupe\sampling.py:39: UserWarning: 35 blocked samples were requested, but only able to sample 32
  % (sample_size, len(blocked_sample)))


### Checking for uncertain pairs

In [ ]:
pair=deduper.uncertainPairs()

In [51]:
pair

[({'dob': '25-10-1953', 'fn': 'WILLIAM', 'gn': 'F', 'ln': 'BLAND JR'},
  {'dob': '25-10-1953', 'fn': 'ROY', 'gn': 'M', 'ln': 'MICHAELSON'})]

In [52]:
labeled_examples = {'match'    : [],
                   'distinct' : pair}

In [53]:
deduper.markPairs(labeled_examples)

### Checking once again for better results (can be repeated as many times as possible)

In [107]:
pair=deduper.uncertainPairs()

In [108]:
pair

[({'dob': '21-02-1962', 'fn': 'WILLIAM', 'gn': 'F', 'ln': 'BLAND JR'},
  {'dob': '08-06-1954', 'fn': 'WILLIAM', 'gn': 'F', 'ln': 'BLAND JR'})]

In [109]:
labeled_examples = {'match'    : ,
                   'distinct' : []}

In [110]:
deduper.markPairs(labeled_examples)

INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (commonTwoTokens, ln), TfidfNGramCanopyPredicate: (0.4, fn))


### Training

In [111]:
deduper.train()

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.000010, score 0.126126126126
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (commonTwoTokens, ln), TfidfNGramCanopyPredicate: (0.4, fn))


In [112]:
with open('trained_labels.json', 'w+') as f:
    deduper.writeTraining(f)
with open('learned_settings', 'wb') as f:
    deduper.writeSettings(f)

In [113]:
threshold = deduper.threshold(data, recall_weight=2)

INFO:dedupe.api:Maximum expected recall and precision
INFO:dedupe.api:recall: 1.000
INFO:dedupe.api:precision: 1.000
INFO:dedupe.api:With threshold: 1.000


In [114]:
duplicates=deduper.match(data,threshold)

In [115]:
clusters=[]
for x in duplicates:
    clusters.append(x[0])

In [116]:
first_elements=[]
duplicates=[]
for cluster in clusters:
    first_elements.append(cluster[0])
    duplicates.extend(list(cluster))


non_duplicates=list(set(range(1,104))-set(duplicates))

### Write output to file

In [124]:
fout=csv.writer(open('sample_output.csv','w+'))
for row_id in data:
    if row_id in non_duplicates or row_id in first_elements:
        fout.writerow(data[row_id].values())